In [33]:
using GAP

mutable struct RubikCube
    state::Vector{Int}
    operators::Dict{String, Any}
    hom::Any
end

function apply_moves!(cube::RubikCube, move_names::Vector{String})
    for m in move_names
        m = strip(m)
        if isempty(m) continue end
        is_inverse = endswith(m, "-") || endswith(m, "'")
        base_move = is_inverse ? m[1:end-1] : m
        
        perm = get(cube.operators, base_move, nothing)
        if perm === nothing continue end
        actual_perm = is_inverse ? inv(perm) : perm
        
        # マッピング計算 (i^P)
        p_map = [Int(i^actual_perm) for i in 1:48]
        
        # 更新ロジックの反転
        next_state = copy(cube.state)
        for i in 1:48
            dest = Int(i^actual_perm)
            next_state[dest] = cube.state[i]
        end
        cube.state = next_state
    end
end

function gap_to_lsystem(word_str::String)
    res = replace(word_str, "*" => " ", "(" => "", ")" => "")
    final_moves = String[]
    for p in split(res)
        m = match(r"([A-Z])(?:\^?([\-\d]+))?", p)
        if m === nothing continue end
        face, pow_str = m.captures[1], m.captures[2]
        pow = pow_str === nothing ? 1 : parse(Int, pow_str)
        pow = pow % 4
        if pow < 0 pow += 4 end
        if pow == 1 push!(final_moves, face)
        elseif pow == 2 (push!(final_moves, face); push!(final_moves, face))
        elseif pow == 3 push!(final_moves, face * "-")
        end
    end
    return final_moves
end

function create_hom(ops)
    names_gap = GAP.evalstr("[\"U\", \"L\", \"F\", \"R\", \"B\", \"D\"]")
    gen_list = [ops[n] for n in ["U", "L", "F", "R", "B", "D"]]
    free = (@gap FreeGroup)(names_gap)
    cube_grp = (@gap Group)(gen_list...)
    return (@gap GroupHomomorphismByImages)(free, cube_grp, (@gap GeneratorsOfGroup)(free), (@gap GeneratorsOfGroup)(cube_grp))
end

function find_solution(cube::RubikCube)
    # 1. 物理状態から置換 g を作成
    # Logic Type-B では、PermList(state) がそのまま「現在の置換 P」を表します。
    gap_list = GAP.evalstr("[" * join(cube.state, ",") * "]")
    g = (@gap PermList)(gap_list)
    
    # 2. 現在の置換 P を打ち消すには、逆元 inv(P) が必要
    # 非常に素直な論理になります。
    word = (@gap PreImagesRepresentative)(cube.hom, inv(g))
    
    word_raw = String((@gap String)(word))
    return gap_to_lsystem(word_raw)
end

# --- 2. 実行・検証セクション ---

operators = Dict(
    "U" => @gap((1,3,8,6)*(2,5,7,4)*(9,33,25,17)*(10,34,26,18)*(11,35,27,19)),
    "L" => @gap((9,11,16,14)*(10,13,15,12)*(1,17,41,40)*(4,20,44,37)*(6,22,46,35)),
    "F" => @gap((17,19,24,22)*(18,21,23,20)*(6,25,43,16)*(7,28,42,13)*(8,30,41,11)),
    "R" => @gap((25,27,32,30)*(26,29,31,28)*(3,38,43,19)*(5,36,45,21)*(8,33,48,24)),
    "B" => @gap((33,35,40,38)*(34,37,39,36)*(3,9,46,32)*(2,12,47,29)*(1,14,48,27)),
    "D" => @gap((41,43,48,46)*(42,45,47,44)*(14,22,30,38)*(15,23,31,39)*(16,24,32,40))
)

cube = RubikCube(collect(1:48), operators, create_hom(operators))

println("1. Scrambling...")
scramble = [rand(["U", "D", "L", "R", "F", "B", "U-", "D-", "L-", "R-", "F-", "B-"]) for _ in 1:20]
println("Scramble: ", join(scramble, " "))
apply_moves!(cube, scramble)

println("\n2. Finding Solution...")
sol = find_solution(cube)
println("Solution: ", join(sol, " "))

println("\n3. Applying Solution...")
apply_moves!(cube, sol)

println("\n--- Result ---")
if cube.state == collect(1:48)
    println("SUCCESS: Cube is perfectly solved!")
else
    println("FAILED: Logic mismatch.")
    println("Mismatch: ", findall(i -> cube.state[i] != i, 1:48))
end


1. Scrambling...
Scramble: B- L L- R- U B- R B R F- D L B- U U- U- R- F- F- L



2. Finding Solution...
Solution: L F U F- U- L- F R U- R- F- L- U L U- L U F U- F- L- F R U R- U- F- U- L U F U- F- L- U- L L B L B- F- L F L U L U- F U- F- L U F L F L- U F F U U F L- D- L- D F- D- B D U U F- B D- B- F D L- F-

3. Applying Solution...

--- Result ---
FAILED: Logic mismatch.
Mismatch: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]


In [34]:
operators = Dict(
    "U" => @gap((1,3,8,6)*(2,5,7,4)*(9,33,25,17)*(10,34,26,18)*(11,35,27,19)),
    "L" => @gap((9,11,16,14)*(10,13,15,12)*(1,17,41,40)*(4,20,44,37)*(6,22,46,35)),
    "F" => @gap((17,19,24,22)*(18,21,23,20)*(6,25,43,16)*(7,28,42,13)*(8,30,41,11)),
    "R" => @gap((25,27,32,30)*(26,29,31,28)*(3,38,43,19)*(5,36,45,21)*(8,33,48,24)),
    "B" => @gap((33,35,40,38)*(34,37,39,36)*(3,9,46,32)*(2,12,47,29)*(1,14,48,27)),
    "D" => @gap((41,43,48,46)*(42,45,47,44)*(14,22,30,38)*(15,23,31,39)*(16,24,32,40))
)

cube = RubikCube(collect(1:48), operators, create_hom(operators))
scramble = ["U", "U-"]
apply_moves!(cube, scramble)
scramble = ["L", "L-"]
apply_moves!(cube, scramble)
scramble = ["F", "F-"]
apply_moves!(cube, scramble)
scramble = ["R", "R-"]
apply_moves!(cube, scramble)
scramble = ["B", "B-"]
apply_moves!(cube, scramble)
scramble = ["D", "D-"]
apply_moves!(cube, scramble)

println("\n--- Result ---")
println("cube.state: ", cube.state)
if cube.state == collect(1:48)
    println("SUCCESS: Cube is perfectly solved!")
else
    println("FAILED: Logic mismatch.")
    println("Mismatch: ", findall(i -> cube.state[i] != i, 1:48))
    println("number of mismatches: ", length(findall(i -> cube.state[i] != i, 1:48)))
end



--- Result ---
cube.state: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
SUCCESS: Cube is perfectly solved!


In [37]:
function find_solution(cube::RubikCube)
    gap_list = GAP.evalstr("[" * join(cube.state, ",") * "]")
    g = (@gap PermList)(gap_list)
    
    word = (@gap PreImagesRepresentative)(cube.hom, g)
    
    word_raw = String((@gap String)(word))
    @show word_raw
    return gap_to_lsystem(word_raw)
end

operators = Dict(
    "U" => @gap((1,3,8,6)*(2,5,7,4)*(9,33,25,17)*(10,34,26,18)*(11,35,27,19)),
    "L" => @gap((9,11,16,14)*(10,13,15,12)*(1,17,41,40)*(4,20,44,37)*(6,22,46,35)),
    "F" => @gap((17,19,24,22)*(18,21,23,20)*(6,25,43,16)*(7,28,42,13)*(8,30,41,11)),
    "R" => @gap((25,27,32,30)*(26,29,31,28)*(3,38,43,19)*(5,36,45,21)*(8,33,48,24)),
    "B" => @gap((33,35,40,38)*(34,37,39,36)*(3,9,46,32)*(2,12,47,29)*(1,14,48,27)),
    "D" => @gap((41,43,48,46)*(42,45,47,44)*(14,22,30,38)*(15,23,31,39)*(16,24,32,40))
)

cube = RubikCube(collect(1:48), operators, create_hom(operators))
scramble = ["U", "L", "F"]
apply_moves!(cube, scramble)

# sol_minimum = ["F-", "L-", "U-"]
# apply_moves!(cube, sol_minimum)

sol = find_solution(cube)
apply_moves!(cube, sol)
@show sol

println("\n--- Result ---")
println("cube.state: ", cube.state)
if cube.state == collect(1:48)
    println("SUCCESS: Cube is perfectly solved!")
else
    println("FAILED: Logic mismatch.")
    println("Mismatch: ", findall(i -> cube.state[i] != i, 1:48))
    println("number of mismatches: ", length(findall(i -> cube.state[i] != i, 1:48)))
end


word_raw = "F^-1*L^-1*U^-1"
sol = ["F-", "L-", "U-"]

--- Result ---
cube.state: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
SUCCESS: Cube is perfectly solved!


In [ ]:
function find_solution(cube::RubikCube)
    gap_list = GAP.evalstr("[" * join(cube.state, ",") * "]")
    g = (@gap PermList)(gap_list)
    
    word = (@gap PreImagesRepresentative)(cube.hom, g)
    
    # --- 修正ポイント：文字列ではなく ExtRep (内部形式) を取得 ---
    # ExtRepOfObj は [gen_idx, pow, gen_idx, pow, ...] という形式のリストを返す
    ext_rep = GAP.Globals.ExtRepOfObj(word)
    @show ext_rep
    
    # GAPのリストをJuliaのVector{Int}に変換
    flat_data = Int[x for x in ext_rep]
    
    return ext_rep_to_moves(flat_data)
end
function ext_rep_to_moves(data::Vector{Int})
    # 生成元のインデックスと名前の対応表（create_hom で定義した順序）
    mapping = Dict(1 => "U", 2 => "L", 3 => "F", 4 => "R", 5 => "B", 6 => "D")
    final_moves = String[]
    # 2つずつ（[インデックス, 指数] のペア）取り出す
    for i in 1:2:length(data)
        idx = data[i]
        pow = data[i+1]
        face = mapping[idx]
        
        # 指数を 0-3 の回転数に正規化
        n = pow % 4
        if n < 0 n += 4 end
        
        if n == 1
            push!(final_moves, face)
        elseif n == 2
            push!(final_moves, face, face) # 2回回す
        elseif n == 3
            push!(final_moves, face * "-") # 逆回転
        end
    end
    return final_moves
end
operators = Dict(
    "U" => @gap((1,3,8,6)*(2,5,7,4)*(9,33,25,17)*(10,34,26,18)*(11,35,27,19)),
    "L" => @gap((9,11,16,14)*(10,13,15,12)*(1,17,41,40)*(4,20,44,37)*(6,22,46,35)),
    "F" => @gap((17,19,24,22)*(18,21,23,20)*(6,25,43,16)*(7,28,42,13)*(8,30,41,11)),
    "R" => @gap((25,27,32,30)*(26,29,31,28)*(3,38,43,19)*(5,36,45,21)*(8,33,48,24)),
    "B" => @gap((33,35,40,38)*(34,37,39,36)*(3,9,46,32)*(2,12,47,29)*(1,14,48,27)),
    "D" => @gap((41,43,48,46)*(42,45,47,44)*(14,22,30,38)*(15,23,31,39)*(16,24,32,40))
)

cube = RubikCube(collect(1:48), operators, create_hom(operators))
scramble = [rand(["U", "D", "L", "R", "F", "B", "U-", "D-", "L-", "R-", "F-", "B-"]) for _ in 1:20]
apply_moves!(cube, scramble)

# sol_minimum = ["F-", "L-", "U-"]
# apply_moves!(cube, sol_minimum)

sol = find_solution(cube)
apply_moves!(cube, sol)
@show sol

println("\n--- Result ---")
println("cube.state: ", cube.state)
if cube.state == collect(1:48)
    println("SUCCESS: Cube is perfectly solved!")
else
    println("FAILED: Logic mismatch.")
    println("Mismatch: ", findall(i -> cube.state[i] != i, 1:48))
    println("number of mismatches: ", length(findall(i -> cube.state[i] != i, 1:48)))
end


ext_rep = GAP: [ 2, -1, 1, -1, 2, 1, 1, 1, 2, 1, 3, -1, 2, -1, 3, -2, 1, 1, 3, -1, 1, -1, 3, -1, 2, 1, 3, 1, 2, -1, 1, 2, 2, -1, 1, -1, 5, 1, 2, -1, 5, -1, 2, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 4, 1, 1, -1, 4, -1, 3, -1, 2, -1, 1, 1, 2, 1, 3, -1, 2, 1, 3, 1, 2, -1, 1, -2, 3, 1, 1, -1, 3, -1, 2, -1, 1, -1, 3, -1, 2, 1, 3, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, -1, 1, 1, 3, 1, 2, 1, 3, 1, 2, -1, 1, 1, 3, -1, 4, 1, 1, 1, 4, -1, 3, 1, 2, 1, 6, 1, 4, 1, 6, -1, 3, -1, 5, 1, 6, -2, 5, -1, 4, -1, 1, -1, 2, -1, 5, -1, 2, 1 ]
sol = ["L-", "U-", "L", "U", "L", "F-", "L-", "F", "F", "U", "F-", "U-", "F-", "L", "F", "L-", "U", "U", "L-", "U-", "B", "L-", "B-", "L", "U", "L", "U", "F", "U", "R", "U-", "R-", "F-", "L-", "U", "L", "F-", "L", "F", "L-", "U", "U", "F", "U-", "F-", "L-", "U-", "F-", "L", "F", "L", "U", "L", "U", "U", "L", "L", "U", "L-", "U", "F", "L", "F", "L-", "U", "F-", "R", "U", "R-", "F", "L", "D", "R", "D-", "F-", "B", "D", "D", "B-", "R-", "U-", "L-", "B-", "L"]

--- Result ---
